In [ ]:
# import graphlab as gl
import matplotlib.pyplot as plt 
# import nltk
import pandas as pd
import numpy as np
# from nltk.corpus import stopwords
from gensim import *
# %matplotlib inline
import matplotlib.colors as mcolors
from matplotlib import *
import spacy, logging, warnings, os, re
from pathlib2 import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import *
import math




In [ ]:
allfiles = os.listdir('../data/categories/')
# allfiles.remove(".DS_Store")
cuisines = [f.replace('.txt','') for f in allfiles]
contents = [Path('../data/categories/'+f).read_text(encoding="utf8")
            .replace('\n',' ') for f in allfiles]

In [ ]:
def vectorizer(content,use_idf = False, max_df = .9):
    vectorizer = TfidfVectorizer(stop_words='english', use_idf=use_idf,max_df=max_df)
    matrix = vectorizer.fit_transform(contents)
    id2words ={}
    for i,word in enumerate(vectorizer.get_feature_names()):
        id2words[i] = word
    return matrix,np.vstack([cosine_similarity(val, matrix)for val in matrix]),id2words

In [ ]:
def draw(matrix):
    fig, ax = plt.subplots()
    cax = ax.matshow(matrix, cmap=cm.YlGnBu)
    ax.set_xticks(np.arange(matrix.shape[1]), minor=False)
    ax.set_yticks(np.arange(matrix.shape[0]), minor=False)
    ax.set_xticklabels(matrix.index, rotation='vertical')
    ax.set_yticklabels(matrix.index)
    fig.colorbar(cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,.8,.90,1])
    fig.set_size_inches(30, 30)
    plt.show()

In [ ]:
matrix_if,similarity_if,_ = vectorizer(contents)
matrix = pd.DataFrame(similarity_if)
matrix.columns = cuisines
matrix.index = cuisines
draw(matrix)

In [ ]:
matrix_ifidf,similarity_ifidf,id2words = vectorizer(contents,True)
matrix = pd.DataFrame(similarity_ifidf)
matrix.columns = cuisines
matrix.index = cuisines
draw(matrix)

In [ ]:

def bracket(ax, pos=[0,0], scalex=1, scaley=1, text="",textkw = {}, linekw = {}):
    x = np.array([0, 0.05, 0.45,0.5])
    y = np.array([0,-0.01,-0.01,-0.02])
    x = np.concatenate((x,x+0.5)) 
    y = np.concatenate((y,y[::-1]))
    ax.plot(x*scalex+pos[0], y*scaley+pos[1], clip_on=False, 
            transform=ax.get_xaxis_transform(), **linekw)
    ax.text(pos[0]+0.5*scalex, (y.min()-0.01)*scaley+pos[1], text, 
                transform=ax.get_xaxis_transform(),
                ha="center", va="top", **textkw)
def draw_with_cluster(matrix, axline):
    fig, ax = plt.subplots()
    cax = ax.matshow(matrix, cmap=cm.YlGnBu)
    ax.set_xticks(np.arange(matrix.shape[1]), minor=False)
    ax.set_yticks(np.arange(matrix.shape[0]), minor=False)
    ax.set_xticklabels(matrix.index, rotation='vertical')
    ax.set_yticklabels(matrix.index)
    for i, line in enumerate(axline):
      plt.axvline(x=line-0.5,color='red', lw=2)
      plt.axhline(y=line-0.5,color='red', lw=2)
      if i ==0:
        bracket(ax, text=i, pos=[0-0.5,-0.01 ], scalex=line-0.5, linekw=dict(color="red", lw=2) )
      else:
        bracket(ax, text=i, pos=[axline[i-1]-0.5,-0.01 ], scalex=axline[i]-axline[i-1]-0.5, linekw=dict(color="red", lw=2) )
    fig.colorbar(cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,.8,.90,1])
    fig.set_size_inches(30, 30)
    plt.show()

In [ ]:
def draw_nclusters(nclusters):
  
  corpus = matutils.Sparse2Corpus(matrix_ifidf, documents_columns=False)
  lda = models.LdaModel(corpus, id2word=id2words, num_topics=nclusters)
  corpus_lda = lda[corpus]
  topics= []
  for i,topic in enumerate(corpus_lda):
    bestCluster = max(topic, key=lambda x:x[1])
    topics.append((i,cuisines[i]+" cluster: "+str(bestCluster[0]),bestCluster[0]))

  clusters = []
  topics.sort(key = lambda x: x[2])
  for topic in topics:
    clusters.append(topic[2])
    
  total = 0
  axlineVals = []
  for cluster in set(clusters):
    total += clusters.count(cluster)
    axlineVals.append(total)
  matrix = pd.DataFrame(similarity_if).reindex([x[0] for x in topics])[[x[0] for x in topics]]
  matrix.columns = [x[1] for x in topics]
  matrix.index = [x[1] for x in topics]
  draw_with_cluster(matrix,axlineVals)


In [ ]:
draw_nclusters(3)

In [ ]:
draw_nclusters(5)

In [ ]:
draw_nclusters(10)

In [ ]:
draw_nclusters(15)

In [ ]:
draw_nclusters(20)

In [ ]:
# corpus = matutils.Sparse2Corpus(matrix_ifidf, documents_columns=False)
# lda = models.LdaModel(corpus, id2word=id2words, num_topics=1)
# corpus_lda = lda[corpus]
# topics= []
# for i,topic in enumerate(corpus_lda):
#   bestCluster = max(topic, key=lambda x:x[1])
#   topics.append((i,cuisines[i]+" cluster: "+str(bestCluster[0]),bestCluster[0]))


In [ ]:
# clusters = []
# topics.sort(key = lambda x: x[2])
# for topic in topics:
#   clusters.append(topic[2])

In [ ]:
# total = 0
# axlineVals = []
# for cluster in set(clusters):
#   total += clusters.count(cluster)
#   axlineVals.append(total)

In [ ]:
# matrix = pd.DataFrame(similarity_if).reindex([x[0] for x in topics])[[x[0] for x in topics]]
# matrix.columns = [x[1] for x in topics]
# matrix.index = [x[1] for x in topics]
# draw_with_cluster(matrix,axlineVals)

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# g=sns.clustermap(df_lda.corr(), center=0, cmap="YlGnBu", metric='cosine', linewidths=.75, figsize=(12, 12))
# plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
# plt.show()